# Logistic Regression Code Along - Titanic

In [1]:
#################### Initialize ###########################

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('myproj').getOrCreate()

# Feature Engineering
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

# Model
from pyspark.ml.classification import LogisticRegression

# Pipeline
from pyspark.ml import Pipeline

# Evaluators
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [2]:
#################### Data ##################################

# Bring in Data
data = spark.read.csv('titanic.csv',inferSchema=True,header=True)
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [4]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [5]:
# Select useful columns
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [6]:
# Drop NA
my_final_data = my_cols.na.drop()

In [7]:
######## Working with Categorical Columns #########

# Convert every string to a number, then one hot encode it. 
# Gender
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

# Embark
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')


In [8]:
# Vectorize this
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [9]:
################### Pipeline ########################

# Setup Model
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

# Setup Pipeline
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

# Split the data
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,.3])

# Fit Model
fit_model = pipeline.fit(train_titanic_data)

# Get results
results = fit_model.transform(test_titanic_data)

# Perform Evaluation
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')
AUC = my_eval.evaluate(results)
AUC

0.7800129785853341